In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

Dòng 2 khởi tạo một spark session<br>
appName: Tên ứng dụng sẽ hiển thị trong spark web UI<br>
SparkSession: cải tiến của sparkcontect giúp việc truy vấn sql dễ dàng hơn<br>

In [2]:
spark = (SparkSession.builder.appName("first spark").getOrCreate())

In [3]:
type(spark)

pyspark.sql.session.SparkSession

Header true thể hiện là dòng đầu tiên của file csv là dòng chứa header <br>
n=10 giới hạn số hàng mà ở đây là 10 hàng <br>
truncate đảm bảo các dữ liệu sẽ không bị cắt ngắn nội dung dòng có dài cả mặt giấy vẫn in ra bằng hết<br>

In [4]:
sales_file = "sales_records.csv"
sale_df = (spark.read.format("csv").option("header","true").load(sales_file))

In [5]:
sale_df.select("Region","Country","Order ID").show(n=10,truncate = False)

+---------------------------------+---------------------+---------+
|Region                           |Country              |Order ID |
+---------------------------------+---------------------+---------+
|Middle East and North Africa     |Azerbaijan           |535113847|
|Central America and the Caribbean|Panama               |874708545|
|Sub-Saharan Africa               |Sao Tome and Principe|854349935|
|Sub-Saharan Africa               |Sao Tome and Principe|892836844|
|Central America and the Caribbean|Belize               |129280602|
|Europe                           |Denmark              |473105037|
|Europe                           |Germany              |754046475|
|Middle East and North Africa     |Turkey               |772153747|
|Europe                           |United Kingdom       |847788178|
|Asia                             |Kazakhstan           |471623599|
+---------------------------------+---------------------+---------+
only showing top 10 rows



tiếp theo sẽ là spark transformation và spark actions<br>
spark có 2 loại transformation đó là narrow transformations và wide transformations<br>
narrow trans có nghĩa là đầu ra chỉ được tính toán từ một đầu vào duy nhất<br>
wide transformation với groupBy và orderBy khiến dữ liệu bị xáo trộn và gộp những dữ liệu giống nhau lại với nhau<br>

.groupBy: mà có 2 cột được chọn thì sẽ nhóm các thuộc tính có cả 2 cột đó giống nhau lại và đếm<br>
.agg(count()): .agg là aggregation có tác dụng gộp lại còn count thì đến số dùng hợp được gộp lại đó
.alias : chúng ta sẽ đếm cột "Order ID" và trả lại số lượng đếm được về một cột mới total orders
.orderBy sắp  xếp lại theo thứ tự từ lớn tới bé

In [6]:
type(sale_df)

pyspark.sql.dataframe.DataFrame

In [7]:
count_sale_df = (sale_df.select("Region","Country","Order ID")
                .groupBy("Region","Country").agg(count("Order ID").alias("Total Orders"))
                .orderBy("Total Orders",ascending = False))
count_sale_df.show(n=10,truncate=False)
print("total row = ",(count_sale_df.count()))

+---------------------------------+------------+------------+
|Region                           |Country     |Total Orders|
+---------------------------------+------------+------------+
|Sub-Saharan Africa               |Sudan       |623         |
|Australia and Oceania            |New Zealand |593         |
|Europe                           |Vatican City|590         |
|Sub-Saharan Africa               |Mozambique  |589         |
|Europe                           |Malta       |589         |
|Middle East and North Africa     |Tunisia     |584         |
|Asia                             |Cambodia    |584         |
|Central America and the Caribbean|Panama      |578         |
|Sub-Saharan Africa               |Rwanda      |576         |
|Sub-Saharan Africa               |South Africa|575         |
+---------------------------------+------------+------------+
only showing top 10 rows

total row =  185


mô hình spark thực thi một cách lười biếng có nghĩa là bạn viết các trans nhưng spark chỉ thực sự chạy khi được gọi <br>
